In [89]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/py_scripts/')
import results
from bs4 import BeautifulSoup
%reload_ext autoreload
%autoreload 2
import requests
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer




In [126]:
model[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4470 entries, 0 to 4469
Data columns (total 13 columns):
Unnamed: 0      4470 non-null int64
Unnamed: 0.1    4470 non-null int64
case_text       4470 non-null object
case_title      4470 non-null object
court           4470 non-null object
date            4470 non-null object
docket          4470 non-null object
tags            4470 non-null object
type_of_law     4470 non-null object
url             4470 non-null object
web_source      4470 non-null object
html_format     4470 non-null object
title_date      4470 non-null object
dtypes: int64(2), object(11)
memory usage: 648.9+ KB


In [7]:
df = pd.read_csv('/Users/kevingmagana/DSI/capstone/case_corpus.csv')
df = df.dropna()
df['title_date'] = df[['case_title', 'date']].apply(lambda x: ' '.join(x), axis=1)


df2 = df.iloc[:, [2, 12]]
df2 = df2.drop_duplicates(subset='title_date')
df2 = df2.reset_index(drop=True)


In [15]:


model = results.load_model()
def process_query(query):
    """
    :param query: string, the query
    :return: list, of indices or DOC IDs
      EX:
      	 [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]
    """
    # Step 1: Intake and get Indexed Cases
    query =  query
    indices = results.indexed_results(model[0], query)
    # Step 2: Get index_vectors and case_val dictionary
    index_vectors, case_value_dictionary= results.get_index_vectors(model[2], indices)
    # Step 3: Vectorize query
    vectorized_query = results.query_vector(model[1], query)
    # Step 4: Get Cosine similarities
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    # Step 5: Get top results and rank them (top 10 right now...)
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)
    res = [case_value_dictionary[array_value] for array_value in top_results[0]]
    return res


In [32]:
model[3].case_text[0]

"United States Court of Appeals,Tenth Circuit.\nAlfredo Yero PORRO, Plaintiff-Appellant, v. Stanley BARNES; and Michael Bryant, in his official capacity as Sheriff of Jefferson County, Defendants-Appellees.\nNo. 10-6002.\nDecided: November 09, 2010\n\nBefore KELLY and GORSUCH, Circuit Judges, and MELGREN, District Judge.**Submitted on the briefs: *Mark E. Bialick and David B. Donchin, Durbin, Larimore & Bialick, Oklahoma City, OK; and David W. Little, Law Offices of David Little, Oklahoma City, OK, for Plaintiff-Appellant. Robert S. Lafferrandre and Randall J. Wood, Pierce Couch Hendrickson Baysinger & Green, L.L.P., Oklahoma City, OK; and Chris J. Collins, Timothy M. Melton, and Linda K. Soper, Collins, Zorn & Wagner, P.C., Oklahoma City, OK, for Defendants-Appellees.\nThis case presents two central questions: What provision of the Constitution should this court use to analyze a federal immigration detainee's claim of excessive force? And does a county's failure to adopt a prophylacti

In [175]:
html = model[3].html_format[4213]

# content = requests.get(html).content
soup = BeautifulSoup(html, 'lxml')


list_of_strings = []
for paragaph in soup.findAll('p'): 
    list_of_strings.append(str(paragaph.text.encode('ascii', 'ignore')).replace('b', '').replace("'", ""))
# print(model[3].html_format[0])

len(list_of_strings)

76

In [108]:
## QUERY: What is the penalty for drug trafficking?
## RESULTS [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]

def get_top_values(lst, n):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    
    [-1:-n-1:-1]
    '''
    return [i for i in np.argsort(lst)[-1:-n-1:-1]]

def most_relevant_part_of_one_text(case, query): 
    
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(case).toarray()
        
    tokenized_query = vectorizer.transform(query)
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)

    return top_results


def tokenize(tokenize_texts):
    """
    :param case_text: list of strings (case_texts)
    :returns: list of strings, tokenized word.
    """
    table = str.maketrans({key: None for key in string.punctuation})
    return [word_tokenize(case.translate(table).lower()) for case in tokenize_texts]

def remove_stop_words(remove_stop_text):
    """
    :param case_text: list of lists of strings (tokens)
    :returns: list of lists of strings(tokens) without stopwords
    """
    stop = set(stopwords.words('english'))
    return [[word for word in words if word not in stop] for words in remove_stop_text]

def stem_words(stem_word_text):
    """
    :param without_stop: list of lists without stop words
    :return: list of lists of strings (stemmed words)
    """
    snowball = SnowballStemmer('english')
    return [[snowball.stem(word) for word in words] for words in stem_word_text]



# most_relevant_part_of_one_text(model[3].case_text[0], query)

# ## Step 

# vectorizer = TfidfVectorizer(stop_words='english')
# vectors = vectorizer.fit_transform(data).toarray()


In [168]:
### Step 1: Vectorize and apply Tf-IDF weights on the case
data= list_of_strings

tokenized = tokenize(data)
removed_stop_words = remove_stop_words(tokenized)
stemmed_words = stem_words(removed_stop_words)
list_of_strings = [" ".join(lst) for lst in stemmed_words]

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(list_of_strings)

vectors.shape, len(data)

((35, 763), 35)

In [169]:
query = ['What is the penalty for drug trafficking?']
### Step 2: Vectorize the query

tokenized = tokenize(query)
removed_stop_words = remove_stop_words(tokenized)
stemmed_words = stem_words(removed_stop_words)
list_of_strings = [" ".join(lst) for lst in stemmed_words]

tokenized_query = vectorizer.transform(list_of_strings)

cosine_similarities = linear_kernel(tokenized_query, vectors) 
top_results = (get_top_values(cosine_similarities[0], 10), query)
print(top_results)

([22, 21, 2, 23, 19, 13, 26, 11, 14, 24], ['What is the penalty for drug trafficking?'])


In [170]:
top_results[0]

[22, 21, 2, 23, 19, 13, 26, 11, 14, 24]

In [174]:
# print(tokenized_query.todense())
# vectorizer.vocabulary_
# print(list_of_strings)
# data= model[3].case_text[2939].split('\n')
print(list_of_strings)
print(data[21])


['penalti drug traffick']
"Sanchez-Garcia also argues that there was insufficient evidence to support his conviction for possession of a firearm in furtherance of a drug trafficking crime.  To secure a conviction under 924(c)(1)(A), the government must present evidence from which a reasonale juror could find a nexus etween the defendants possession of the charged firearm and the drug crime, such that this possession had the effect of furthering, advancing or helping forward the drug crime. United States v. Hamilton, 332 F.3d 1144, 1149-50 (8th Cir.2003).  Accordingly, evidence that the defendant simultaneously possessed drugs and a firearm, standing alone, would not warrant sumitting the charge to the jury. United States v. Spencer, 439 F.3d 905, 914 (8th Cir.2006) (quoting Hamilton, 332 F.3d at 1150).  Instead, the jury must e ale to infer that the defendants possession of the firearm facilitated the drug crime, through evidence that the firearm was used for protection, was kept near 

In [124]:
vectorizer.vocabulary_

{'09': 0,
 '100000': 1,
 '103': 2,
 '104': 3,
 '1043': 4,
 '105': 5,
 '106002': 6,
 '109': 7,
 '10th': 8,
 '110': 9,
 '112': 10,
 '114': 11,
 '1146': 12,
 '115': 13,
 '1151': 14,
 '1155': 15,
 '1160': 16,
 '117': 17,
 '118': 18,
 '1185': 19,
 '1197': 20,
 '11th': 21,
 '1202': 22,
 '1204': 23,
 '1210': 24,
 '1227': 25,
 '1230': 26,
 '1243': 27,
 '1244': 28,
 '1254': 29,
 '1270': 30,
 '1278': 31,
 '129': 32,
 '1299': 33,
 '1307': 34,
 '132': 35,
 '1341': 36,
 '1346': 37,
 '137': 38,
 '14': 39,
 '140': 40,
 '1401': 41,
 '143': 42,
 '15': 43,
 '156': 44,
 '159': 45,
 '16': 46,
 '16566': 47,
 '1708': 48,
 '172': 49,
 '173': 50,
 '1865': 51,
 '1937': 52,
 '1949': 53,
 '1977and': 54,
 '1978': 55,
 '1979': 56,
 '1983': 57,
 '1983s': 58,
 '1983to': 59,
 '1985': 60,
 '1989': 61,
 '1989mr': 62,
 '1992': 63,
 '1995': 64,
 '1995meanwhil': 65,
 '1998': 66,
 '2009': 67,
 '2010': 68,
 '2010bthat': 69,
 '2018': 70,
 '209': 71,
 '2151': 72,
 '238': 73,
 '2689': 74,
 '272': 75,
 '2732': 76,
 '304': 77,
 

In [56]:
data[5]

'Before KELLY and GORSUCH, Circuit Judges, and MELGREN, District Judge.**Submitted on the briefs: *Mark E. Bialick and David B. Donchin, Durbin, Larimore & Bialick, Oklahoma City, OK; and David W. Little, Law Offices of David Little, Oklahoma City, OK, for Plaintiff-Appellant. Robert S. Lafferrandre and Randall J. Wood, Pierce Couch Hendrickson Baysinger & Green, L.L.P., Oklahoma City, OK; and Chris J. Collins, Timothy M. Melton, and Linda K. Soper, Collins, Zorn & Wagner, P.C., Oklahoma City, OK, for Defendants-Appellees.'